In [1]:
%matplotlib inline

import utils_ted
from utils_ted import *

Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

## Setup

We haven't really looked into the detail of how this works yet - so this is provided for self-study for those who are interested. We'll look at it closely next week.

In [3]:
path=get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path, encoding='utf8').read().lower()


606208/600901 [==============================] - 7s 12us/step


In [4]:
print('corpus length:', len(text))

corpus length: 600893


In [5]:
!tail {path} -n25

are thinkers who believe in the saints.


144

It stands to reason that this sketch of the saint, made upon the model
of the whole species, can be confronted with many opposing sketches that
would create a more agreeable impression. There are certain exceptions
among the species who distinguish themselves either by especial
gentleness or especial humanity, and perhaps by the strength of their
own personality. Others are in the highest degree fascinating because
certain of their delusions shed a particular glow over their whole
being, as is the case with the founder of christianity who took himself
for the only begotten son of God and hence felt himself sinless; so that
through his imagination--that should not be too harshly judged since the
whole of antiquity swarmed with sons of god--he attained the same goal,
the sense of complete sinlessness, complete irresponsibility, that can
now be attained by every individual through science.--In the same manner
I have viewed the saints of India

In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars) + 1

In [7]:
print("total chars : %s" % vocab_size)

total chars : 58


In [8]:
chars.insert(0, '/n')

In [9]:
"".join(chars[1:-4])

'\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyz'

In [10]:
char_indices = {c:i for i, c in enumerate(chars)}
indices_char = {i:c for i, c in enumerate(chars)}

In [11]:
text_idxs = [char_indices[c] for c in text]

In [12]:
print(text_idxs[:10])

[43, 45, 32, 33, 28, 30, 32, 1, 1, 1]


In [13]:
''.join(indices_char[idx] for idx in text_idxs[:70])

'preface\n\n\nsupposing that truth is a woman--what then? is there not gro'

## Preprocess and create model

In [14]:
maxlen = 40
sentences_list = []
next_chars_list = []
for i in range(0, (len(text_idxs) - maxlen + 1)):
    sentences_list.append(text_idxs[i: i+maxlen])
    next_chars_list.append(text_idxs[i+1: i+maxlen+1])

In [15]:
print("nb sequences:", len(sentences_list))

nb sequences: 600854


In [16]:
sentences = np.array(sentences_list[:-2])
next_chars = np.array(next_chars_list[:-2])

In [17]:
print(sentences.shape, next_chars.shape)

(600852, 40) (600852, 40)


In [18]:
n_fac = 24

In [19]:
from keras.layers import LSTM

[Keras 2.0 release notes](https://github.com/fchollet/keras/wiki/Keras-2.0-release-notes)

```
Recurrent layers
    output_dim -> units
    init -> kernel_initializer
    inner_init -> recurrent_initializer
    added argument bias_initializer
    W_regularizer -> kernel_regularizer
    b_regularizer -> bias_regularizer
    added arguments kernel_constraint, recurrent_constraint, bias_constraint
    dropout_W -> dropout
    dropout_U -> recurrent_dropout
    consume_less -> implementation. String values have been replaced with integers: implementation 0 (default), 1 or 2.
    LSTM only: the argument forget_bias_init has been removed. Instead there is a boolean argument unit_forget_bias, defaulting to True.
```

In [20]:
model = Sequential([
    Embedding(vocab_size, n_fac, input_length=maxlen),
    LSTM(512, input_dim=n_fac, return_sequences=True, 
         dropout=0.2, recurrent_dropout=0.2, implementation=1),
    Dropout(0.2),
    LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, implementation=1),
    Dropout(0.2),
    TimeDistributed(Dense(vocab_size)),
    Activation('softmax')    
])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  after removing the cwd from sys.path.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, implementation=1, input_shape=(None, 24))`
  after removing the cwd from sys.path.


In [21]:
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy')

## Train

In [25]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(200):
        x = np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = np.squeeze(model.predict(x, verbose=0))[-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [23]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=1, verbose=2)

Epoch 1/1

  6912/600852 [..............................] - ETA: 36:52 - loss: 3.1599


 13888/600852 [..............................] - ETA: 32:47 - loss: 3.0403


 20864/600852 [>.............................] - ETA: 31:14 - loss: 2.8994


 27840/600852 [>.............................] - ETA: 30:18 - loss: 2.7644


 34816/600852 [>.............................] - ETA: 29:35 - loss: 2.6545


...

573568/600852 [===========================>..] - ETA: 1:21 - loss: 1.5136


580608/600852 [===========================>..] - ETA: 1:00 - loss: 1.5111


587776/600852 [============================>.] - ETA: 39s - loss: 1.5087


594944/600852 [============================>.] - ETA: 17s - loss: 1.5062


600852/600852 [==============================] - 1795s 3ms/step - loss: 1.5042


In [26]:
print_example()

ethics is a basic foundation of all that depression sexbal surmounting
of the same tyranny for his development and
own yeess. a delight by the highest defensency? the world, this last
"free spirits"!--how could: he concerns his party and as


In [27]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=1, verbose=2)

Epoch 1/1
 - 1771s - loss: 1.2742


In [28]:
print_example()

ethics is a basic foundation of all that there is a rank is political religion, and
morality and rigour spirit renders the
causes of
others. they can find ourselves?

144. what does it not follow growth, as anything
else!
but
does not thus



In [29]:
model.save_weights('../data/char_rnn1.h5')